In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import os
import sys
import time
import math
import torch.nn.init as init

In [3]:
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import os
import argparse

In [4]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [5]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential(
            nn.Conv2d(in_planes, self.expansion*planes,
                      kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(self.expansion*planes))


    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [6]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [7]:
"""
block: Block type, we have two types of base block.
       BasicBlock which is designed in ResNet-18.
       Bottleneck which is designed in ResNet-50.
num_blocks: the number of each layer in the block
channels: The size of each filter.
          channels[0]: Convl_CH
          channels[1]: Layer1_CH
          channels[2]: Layer2_CH
          channels[3]: Layer3_CH
"""
class ResNet_three_layers(nn.Module):
    def __init__(self, block, num_blocks, channels, num_classes=10):
        super(ResNet_three_layers, self).__init__()
        self.in_planes = channels[0]

        self.conv1 = nn.Conv2d(3, channels[0], kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels[0])
        self.layer1 = self._make_layer(block, channels[1], num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, channels[2], num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, channels[3], num_blocks[2], stride=2)
        self.linear = nn.Linear(channels[-1]*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [8]:
"""
block: Block type, we have two types of base block.
       BasicBlock which is designed in ResNet-18.
       Bottleneck which is designed in ResNet-50.
num_blocks: the number of each layer in the block
channels: The size of each filter.
          channels[0]: Convl_CH
          channels[1]: Layer1_CH
          channels[2]: Layer2_CH
          channels[3]: Layer3_CH
          channels[4]: Layer4_CH
"""
class ResNet_four_layers(nn.Module):
    def __init__(self, block, num_blocks, channels, num_classes=10):
        super(ResNet_four_layers, self).__init__()
        self.in_planes = channels[0]

        self.conv1 = nn.Conv2d(3, channels[0], kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels[0])
        self.layer1 = self._make_layer(block, channels[1], num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, channels[2], num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, channels[3], num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, channels[4], num_blocks[3], stride=2)
        self.linear = nn.Linear(channels[-1]*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [9]:
def ResNet_Custom():
    """
    block:{BasicBlock, Bottleneck}
    num_blocks: int list with length 3
    we have tried [2, 6 ,2], [2, 8, 2], [3, 3, 3], [5, 5, 5], [7, 7, 7]
    channels: int list with length 4
    we have tried [16, 16, 32, 64], [32, 32, 64, 128], [64, 64, 128, 128]
    """
    # return ResNet_three_layers(BasicBlock, [7, 7, 7], [16, 16, 32, 64])

    """
    block:{BasicBlock, Bottleneck}
    num_blocks: int list with length 4
    we have tried [2, 2, 2, 2], [2, 2, 4, 4], [4, 4, 2, 2], [3, 4, 6, 3], [3, 4, 5, 3]
    channels: int list with length 5
    we have tried [16, 32, 64, 128, 256], [32, 32, 64, 128, 256], [16, 16, 32, 64, 128], [32, 128, 128, 128, 256], [32, 64, 128, 128, 256]
    """
    return ResNet_four_layers(BasicBlock, [2, 2, 2, 2], [16, 32, 64, 128, 256])

In [10]:
print('==> Building model..')
net = ResNet_Custom()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

==> Building model..


In [11]:
pip install torch-summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
from torchsummary import summary

In [13]:
summary(net, (3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
├─ResNet_four_layers: 1-1                [-1, 10]                  --
|    └─Conv2d: 2-1                       [-1, 16, 32, 32]          432
|    └─BatchNorm2d: 2-2                  [-1, 16, 32, 32]          32
|    └─Sequential: 2-3                   [-1, 32, 32, 32]          --
|    |    └─BasicBlock: 3-1              [-1, 32, 32, 32]          14,528
|    |    └─BasicBlock: 3-2              [-1, 32, 32, 32]          19,648
|    └─Sequential: 2-4                   [-1, 64, 16, 16]          --
|    |    └─BasicBlock: 3-3              [-1, 64, 16, 16]          57,728
|    |    └─BasicBlock: 3-4              [-1, 64, 16, 16]          78,208
|    └─Sequential: 2-5                   [-1, 128, 8, 8]           --
|    |    └─BasicBlock: 3-5              [-1, 128, 8, 8]           230,144
|    |    └─BasicBlock: 3-6              [-1, 128, 8, 8]           312,064
|    └─Sequential: 2-6                   [-1, 256, 4, 4]  

Layer (type:depth-idx)                   Output Shape              Param #
├─ResNet_four_layers: 1-1                [-1, 10]                  --
|    └─Conv2d: 2-1                       [-1, 16, 32, 32]          432
|    └─BatchNorm2d: 2-2                  [-1, 16, 32, 32]          32
|    └─Sequential: 2-3                   [-1, 32, 32, 32]          --
|    |    └─BasicBlock: 3-1              [-1, 32, 32, 32]          14,528
|    |    └─BasicBlock: 3-2              [-1, 32, 32, 32]          19,648
|    └─Sequential: 2-4                   [-1, 64, 16, 16]          --
|    |    └─BasicBlock: 3-3              [-1, 64, 16, 16]          57,728
|    |    └─BasicBlock: 3-4              [-1, 64, 16, 16]          78,208
|    └─Sequential: 2-5                   [-1, 128, 8, 8]           --
|    |    └─BasicBlock: 3-5              [-1, 128, 8, 8]           230,144
|    |    └─BasicBlock: 3-6              [-1, 128, 8, 8]           312,064
|    └─Sequential: 2-6                   [-1, 256, 4, 4]  

In [14]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [16]:
# Training
def train(epoch):
    # print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        '''
        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        '''

In [17]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            '''
            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
            '''

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        # print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt1.pth')
        best_acc = acc
        print(best_acc)

In [18]:
from tqdm import tqdm

In [19]:
for epoch in tqdm(range(start_epoch, start_epoch+200)):
    train(epoch)
    test(epoch)
    scheduler.step()

  0%|          | 1/200 [00:19<1:04:48, 19.54s/it]

48.76


  1%|          | 2/200 [00:38<1:02:28, 18.93s/it]

57.44


  2%|▏         | 3/200 [00:56<1:01:47, 18.82s/it]

...

 96%|█████████▋| 193/200 [1:01:36<02:14, 19.15s/it]

94.9


 97%|█████████▋| 194/200 [1:01:55<01:54, 19.15s/it]

94.93


100%|██████████| 200/200 [1:03:50<00:00, 19.15s/it]
